# <h1>BIG DATA ASSIGNMENT WEEK 09 - Collaborative Filtering</h1>

*   Nama : Benedictus Bimo C W
*   Student ID : 5025201097
*   Class : Big Data A

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 21.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=24bf2b721cf7be7ee31e683afdff8cdd32b59ff0311ce0ed540a4faa5d139fdd
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

In [3]:
# Create a SparkSession
spark = SparkSession.builder \
        .master("local") \
        .appName("myApp") \
        .config("spark.sql.session.timeZone", "UTC") \
        .config("spark.sql.session.timeout", "48h") \
        .getOrCreate()

In [4]:
lines = spark.read.text("./sample_data/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

## Build Recomendation model using ALS

In [5]:
max_iters = [5, 10, 20]
reg_params = [0.1, 0.5, 1.0]

results = {}

Pada bagian ini, saya menambahkan variabel dengan 3 nilai berbeda dan params dengan 3 nilai berbeda yang akan digunakan.

In [6]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
for max_iter in max_iters:
    for reg_param in reg_params:
        als = ALS(maxIter=max_iter, regParam=reg_param, userCol="userId", itemCol="movieId", ratingCol="rating",
                  coldStartStrategy="drop")
        model = als.fit(training)

        predictions = model.transform(test)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")
        rmse = evaluator.evaluate(predictions)

        results[(max_iter, reg_param)] = rmse
        print(f"Root-mean-square error for maxIter={max_iter}, regParam={reg_param} = {rmse}")

Root-mean-square error for maxIter=5, regParam=0.1 = 0.9724204860082384
Root-mean-square error for maxIter=5, regParam=0.5 = 1.223479671191816
Root-mean-square error for maxIter=5, regParam=1.0 = 1.529178557016268
Root-mean-square error for maxIter=10, regParam=0.1 = 0.9314489707984631
Root-mean-square error for maxIter=10, regParam=0.5 = 1.222495315592619
Root-mean-square error for maxIter=10, regParam=1.0 = 1.5291809520926887
Root-mean-square error for maxIter=20, regParam=0.1 = 0.9259429927483817
Root-mean-square error for maxIter=20, regParam=0.5 = 1.2223256285741142
Root-mean-square error for maxIter=20, regParam=1.0 = 1.5291809622717731


Dengan referensi kode yang sudah ada, saya menambahkan perulangan untuk *max_iters* *reg_params*.

In [7]:
best_params = min(results, key=results.get)
best_rmse = results[best_params]
print(f"\nBest hyperparameters: maxIter={best_params[0]}, regParam={best_params[1]} with RMSE={best_rmse}")


Best hyperparameters: maxIter=20, regParam=0.1 with RMSE=0.9259429927483817


Dari proses Build Recommendation model dengan ALS, didapatkan hasil akhir bahwa *hyperparameters* terbaik ada di maxIter=20, regParam=0.1 dengan RMSE=0.9832103291575914

## Generate Movie Recomendation

In [8]:
als = ALS(maxIter=best_params[0], regParam=best_params[1], userCol="userId", itemCol="movieId", ratingCol="rating",
               coldStartStrategy="drop")
model = als.fit(training)

## Results

In [10]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{94, 2.7321758},...|
|    10|[{92, 3.4005594},...|
|     0|[{92, 3.0807102},...|
|     1|[{62, 3.1435}, {6...|
|    21|[{29, 4.455954}, ...|
|    11|[{27, 4.81987}, {...|
|    12|[{46, 4.520196}, ...|
|    22|[{88, 4.5953846},...|
|     2|[{93, 4.562735}, ...|
|    13|[{93, 3.2150369},...|
|     3|[{51, 4.1657543},...|
|    23|[{32, 4.4042273},...|
|     4|[{29, 3.431337}, ...|
|    24|[{69, 4.547197}, ...|
|    14|[{52, 4.4557896},...|
|     5|[{55, 3.7784233},...|
|    15|[{46, 4.0075283},...|
|    25|[{71, 2.6936715},...|
|    26|[{22, 4.5666056},...|
|     6|[{25, 4.0230966},...|
+------+--------------------+
only showing top 20 rows



In [11]:
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 3.8925557},...|
|     40|[{28, 3.5408556},...|
|     10|[{23, 3.5163057},...|
|     50|[{23, 3.7053907},...|
|     80|[{26, 2.9694862},...|
|     70|[{21, 3.0953777},...|
|     60|[{22, 2.5950842},...|
|     90|[{17, 4.767898}, ...|
|     30|[{24, 4.298145}, ...|
|      0|[{28, 2.711257}, ...|
|     31|[{12, 3.0547762},...|
|     81|[{11, 3.4582436},...|
|     91|[{12, 2.7460687},...|
|      1|[{12, 3.3062055},...|
|     41|[{4, 3.0939305}, ...|
|     61|[{16, 2.1253796},...|
|     51|[{26, 4.4441185},...|
|     21|[{26, 2.813866}, ...|
|     11|[{18, 3.393278}, ...|
|     71|[{12, 2.8515482},...|
+-------+--------------------+
only showing top 20 rows



In [12]:
# Generate top 10 movie recommendations for a specific set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{22, 4.5666056},...|
|    19|[{90, 3.445723}, ...|
|    29|[{90, 3.3752584},...|
+------+--------------------+



In [13]:
# Generate top 10 user recommendations for a specific set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     65|[{23, 4.125318}, ...|
|     26|[{28, 2.378064}, ...|
|     29|[{21, 4.455954}, ...|
+-------+--------------------+



## Summary

The steps I took to analyze the MovieLens dataset are:
1. Load the dataset and preprocessed it, creating training and test sets.
2. Tested different combinations of *maxIter* and *regParam*, evaluated the models on the test set, and stored RMSE results in a dictionary.
3. Chose the best hyperparameter combination based on the lowest RMSE and displayed the results.
4. Trained the model using the best hyperparameters and generated recommendations for all users, all movies, a subset of users, and a subset of movies.
5. Lastly, display the top 10 recommendations for each user, movie, and the specified subsets in the output.